In [1]:
import numpy as np
import functools as ft
import itertools
import os
from nestedforloop import get_iterator

In [2]:
qubit_number=4
Pauli=np.asarray([
        [[1,0],
        [0,1]],


        [[0,1],
        [1,0]],


        [[0,-1j],
        [1j,0]],


        [[1,0],
        [0,-1]]])


states=np.asarray([
    [1,1]/np.sqrt(2),
    [1,-1]/np.sqrt(2),
    [1,1j]/np.sqrt(2),
    [1,-1j]/np.sqrt(2),
    [1,0],
    [0,1],
])

In [ ]:
def get_proj_iterator(N_qubit, basis):
    d={"x": [[1,1]/np.sqrt(2),
             [1,-1]/np.sqrt(2)],
       "y": [[1,1j]/np.sqrt(2),
            [1,-1j]/np.sqrt(2)],
       "z": [[1,0],[0,1]]
      }
    
    basis_order = get_iterator(3, N_qubit)
    projectors_order = get_iterator(2, N_qubit)
    columns=[]
    
    
    character_matrix=[]
    for i in basis_order:
        string=""
        for j in i:
            string=string+basis[j]
        lines=[]    
        for k in projectors_order:
            o=0
            vect=[]
            for n in k:
                vect.append(d[string[o]][n])
                o=o+1
            lines.append(vect)    
        character_matrix.append(lines)         
        columns.append(string)   
        
    return character_matrix 

In [3]:
import numpy as np
import itertools
import functools as ft

basis=["x","y","z"]
v = get_proj_iterator(4, basis)


projectors2 = []
v=np.reshape(v,[3**qubit_number*2**qubit_number,qubit_number,2])
for i in list(v):
    projectors2.append(np.outer(ft.reduce(np.kron, i),np.conjugate(np.transpose(ft.reduce(np.kron, i)))))
    
    
observables = []
perm = itertools.product(Pauli, repeat=qubit_number)
for i in list(perm):
    observables.append(ft.reduce(np.kron, i)) 

psi=np.zeros((len(observables[1:]),len(projectors2)))
# psi_aux=np.zeros((3*qubit_number,2**qubit_number,4*qubit_number-1))

for i in range(len(observables[1:])):
    for j in range(len(projectors2)):
        psi[i][j]=np.trace(projectors2[j]@observables[i+1])/(2**qubit_number)
psi_aux=np.reshape(np.transpose(psi),[3**qubit_number,2**qubit_number,4**qubit_number-1]) 

invPSI=np.linalg.inv(psi@psi.transpose())    

C:\Users\LauraMartins\AppData\Local\Temp\ipykernel_20752\4134055971.py:50: ComplexWarning: Casting complex values to real discards the imaginary part
  psi[i][j]=np.trace(projectors2[j]@observables[i+1])/(2**qubit_number)


In [4]:
working_dir = os.getcwd()
np.save(working_dir+'\SavedVariables\X_matrix'+str(qubit_number), psi_aux)
np.save(working_dir+'\SavedVariables\invXtX_matrix'+str(qubit_number), invPSI)

In [28]:
# m=np.load(working_dir+'\SavedVariables\X_matrix2.npy')
# print(np.round(psi_aux-m,3))